Analyse the time to live of RPKI objects by extension

In [1]:
from sqlalchemy import create_engine, text

import pandas as pd

engine = create_engine('postgresql://localhost:5432/delta')

with engine.connect() as conn:
    df = pd.read_sql_query(text("""select split_part(uri, '.', -1) as extension, (disappearedon - visibleon) as lifetime_ms from objects where disappearedon is not null and disappearedon < 1000*EXTRACT(EPOCH FROM NOW());"""), conn)

In [40]:
df['duration'] = pd.to_timedelta(df.lifetime_ms, unit='ms')
filtered = (df.extension != 'tal')
df_duration_days = df[filtered].groupby(['extension', pd.Grouper(key='duration', freq="1h")]).count()

In [72]:
import altair as alt

df_viz = df_duration_days.reset_index()
df_viz['hours'] = df_viz.duration.dt.total_seconds() / 3600
df_viz = df_viz[df_viz.hours < 50]

In [73]:


alt.Chart(df_viz[['hours', 'lifetime_ms', 'extension']]).mark_bar().encode(
    x=alt.X('hours:O'),
    y=alt.Y('lifetime_ms:Q'),
    color='extension:N',
)

alt.Chart(...)

In [34]:
s =alt.Y('lifetime_ms')


Undefined